In [10]:
import numpy as np

# Factoraisation LU

## Algorithme de crout

In [11]:
mat = np.random.randint(0, 9, (4, 4))
mat

array([[2, 1, 1, 5],
       [6, 0, 4, 0],
       [4, 8, 4, 0],
       [0, 7, 4, 0]])

In [12]:
def decomposition_crout(M : np.ndarray):
    n, m = M.shape
    assert n==m
    L = np.eye(n)
    U = np.zeros((n,n))
    U [0,:] = M[0, :]
    L[1:, 0] = M[1:, 0]*1/U[0, 0]
    for k in range(1, n-1):
        U[k, k:] = M[k, k:] - L[k, :k]@U[:k, k:]
        L[k+1:, k] = (1/U[k, k])*(M[k+1:,k] - L[k+1:, :k]@U[:k, k])
    U[n-1, n-1] = M[n-1,n-1] -  L[n-1, :n-1]@U[:n-1, n-1]
    return L, U


L, U  = decomposition_crout(mat)

print(L)
print(U)


[[ 1.          0.          0.          0.        ]
 [ 3.          1.          0.          0.        ]
 [ 2.         -2.          1.          0.        ]
 [ 0.         -2.33333333  1.58333333  1.        ]]
[[  2.           1.           1.           5.        ]
 [  0.          -3.           1.         -15.        ]
 [  0.           0.           4.         -40.        ]
 [  0.           0.           0.          28.33333333]]


In [13]:
L@U


array([[2., 1., 1., 5.],
       [6., 0., 4., 0.],
       [4., 8., 4., 0.],
       [0., 7., 4., 0.]])

# Résolution de systeme

In [14]:
def my_up(A, b):
    assert A.shape[0] == b.shape[0]
    n = b.shape[0]
    x= np.zeros((n,))
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.sum(A[i, i+1:]*x[i+1:]))/A[i, i]

In [15]:
generate_h = lambda n : np.fromfunction(lambda i, j: 1/(1+ i + j ), (n, n), dtype=int)
n = [2, 5, 10, 20, 50]

def solve_upper(A, b):
    # verifing matrix sizes
    assert A.shape[0] == b.shape[0]
    n = b.shape[0]
    x= np.zeros((n,))
    # we compute the last value out of our loop to avoid out of index error during looping
    x[n-1] = b[n-1]/A[n-1, n-1]
    for i in range(n-2, -1, -1):
        x[i] = (b[i] - np.sum(A[i, i+1:]*x[i+1:]))/A[i, i]
    return x

def solve_lower(A, b):
    assert A.shape[0] == b.shape[0]
    n = b.shape[0]
    x= np.zeros((n,))
    x[0] = b[0]/A[0, 0]
    for i in range(1, n):
        x[i] = (b[i] - np.sum(A[i, :i]*x[:i]))/A[i, i]
    return x


def solve_Hn(H):
    n = H.shape[0]
    L, U = decomposition_crout(H)
    ones = np.ones((n, ))
    y = solve_lower(L, ones)
    x = solve_upper(U, y)
    return x

H_matrix = [generate_h(i) for i in n]


for i in range(len(n)):
    print(n[i] , solve_Hn(H_matrix[i]))
    



2 [-2.  6.]
5 [    5.  -120.   630. -1120.   630.]
10 [-9.99772302e+00  9.89801913e+02 -2.37557593e+04  2.40201295e+05
 -1.26107480e+06  3.78326957e+06 -6.72588075e+06  7.00046751e+06
 -3.93779280e+06  9.23685928e+05]
20 [-3.47644485e+01  5.11015614e+03 -1.71615984e+05  2.14376407e+06
 -9.31258250e+06 -2.74211830e+07  4.59111316e+08 -2.05689520e+09
  4.52030749e+09 -4.37342307e+09 -5.02796937e+08  2.45934298e+09
  5.85924129e+09 -1.21170891e+10  2.26396465e+09  9.70803574e+09
 -6.37877562e+09 -2.43050196e+09  3.60088806e+09 -9.76652983e+08]
50 [-3.86662120e+01  6.06433923e+03 -2.32625078e+05  3.77435000e+06
 -3.16621549e+07  1.48410209e+08 -3.83113237e+08  4.34129658e+08
  1.91108970e+08 -9.03218443e+08  1.19485025e+08  1.52169005e+09
 -1.51062387e+09  5.61528149e+08 -7.54709098e+08  7.24346027e+08
  3.15161122e+08 -1.92398157e+08 -1.15135476e+09  1.28918603e+09
 -3.88156605e+08 -3.28135602e+08  5.83224818e+08  8.95328811e+08
 -1.06420147e+09 -5.64525295e+08 -8.64135507e+08  9.61806685

# Factorisation de cholesky

In [16]:
def cholesky(M):
    n = M.shape[0]
    B = np.zeros((n,n))
    B[0, 0] = np.sqrt(M[0, 0])
    B[1:, 0] = (1/B[0, 0])*M[1:, 0]
    B[0, 1:] = B[1:, 0]
    for k in range(1, n-1):
        B[k, k] = np.sqrt(M[k, k] - B[k, :k]@B[:k, k])
        B[k+1:, k] = (1/B[k, k])*(M[k+1:, k] - B[k+1:, :k]@B[:k, k])
        B[k, k+1:] = B[k+1:, k]
    B[n-1, n-1]= np.sqrt(M[n-1, n-1] - B[n-1, :n-1]@B[:n-1, n-1])
    return B

h5 = H_matrix[2]

A = .5*(h5 + h5.T)
    
B = cholesky(A)


ones = np.ones((h5.shape[0], ))
y = solve_lower(B, ones)
x = solve_upper(B.T, y)

print(n[2], x)

10 [-9.99745732e+00  9.89781648e+02 -2.37553709e+04  2.40198073e+05
 -1.26106063e+06  3.78323333e+06 -6.72582503e+06  7.00041675e+06
 -3.93776755e+06  9.23680645e+05]


In [17]:
np.linalg.solve(A, ones)

array([-9.99982337e+00,  9.89984621e+02, -2.37596706e+04,  2.40236992e+05,
       -1.26124560e+06,  3.78374027e+06, -6.72665461e+06,  7.00121663e+06,
       -3.93818664e+06,  9.23772647e+05])